In [1]:
%load_ext lab_black

In [2]:
from datetime import date, timedelta
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
df_btc = pd.read_csv("gemini_BTCUSD_day.csv", header=1)
df_eth = pd.read_csv("gemini_ETHUSD_day.csv", header=1)
df_sol = pd.read_csv("solana_investingcom.csv", header=0)

currency = pd.read_csv("FRB.csv")
sp500 = pd.read_csv("HistoricalData_spx.csv")
nasdaq = pd.read_csv("HistoricalData_comp.csv")

### preprocessing 

In [4]:
def preprocess_gemini_data(df):
    df.columns = df.columns.str.lower().str.replace(" ", "_")
    df = df[["date", "symbol", "open", "high", "low", "close", "volume"]].copy()
    df.date = pd.to_datetime(df.date).dt.date
    df.index = df.date
    df = df.drop(columns="date")
    return df


def preprocess_solana(df):
    df.columns = df.columns.str.lower()
    df.date = pd.to_datetime(df.date)
    df.index = df.date
    return df


def preprocess_currency_data_to_series(currency, col_number):
    euro = currency.iloc[6:, [0, col_number]]
    euro.columns = ["date", "price"]
    euro.date = pd.to_datetime(euro.date, errors="coerce")
    euro.price = pd.to_numeric(euro.price, errors="coerce")
    euro.index = euro.date
    euro = euro.price.dropna()
    return euro


def preprocess_ns_data_to_series(df):
    s = df[["Date", "Open"]]
    s.columns = s.columns.str.lower()
    s.index = pd.to_datetime(s.date)
    s = s.open
    return s

In [5]:
df_btc = preprocess_gemini_data(df_btc)
df_eth = preprocess_gemini_data(df_eth)
df_sol = preprocess_solana(df_sol)

btc = df_btc.open
eth = df_eth.open
sol = df_sol.open

sp500 = preprocess_ns_data_to_series(sp500)
nasdaq = preprocess_ns_data_to_series(nasdaq)
currency = preprocess_currency_data_to_series(currency, 2)

In [6]:
ana = pd.concat(
    [
        btc.rename("btc"),
        eth.rename("eth"),
        sol.rename("sol"),
        sp500.rename("sp500"),
        nasdaq.rename("nas"),
        currency.rename("eur"),
    ],
    axis=1,
    join="outer",
)
# ana = ana.dropna(axis=0)[1:].copy()

/opt/homebrew/Caskroom/miniforge/base/envs/pytorch/lib/python3.8/site-packages/pandas/core/indexes/base.py:3512: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  indexer = self._engine.get_indexer(target._get_engine_target())
/opt/homebrew/Caskroom/miniforge/base/envs/pytorch/lib/python3.8/site-packages/pandas/core/algorithms.py:1799: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  sorter = values.argsort()
/opt/homebrew/Caskroom/miniforge/base/envs/pytorch/lib/python3.8/site-packages/pandas/core/indexes/base.py:1963: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in orde

In [7]:
ana.corr()

,btc,eth,sol,sp500,nas,eur
btc,1.000000,0.927875,0.649573,0.899885,0.904437,0.346348
eth,0.927875,1.000000,0.899620,0.863744,0.836705,0.314957
sol,0.649573,0.899620,1.000000,0.809123,0.775379,-0.703472
sp500,0.899885,0.863744,0.809123,1.000000,0.986500,0.255733
nas,0.904437,0.836705,0.775379,0.986500,1.000000,0.304599
eur,0.346348,0.314957,-0.703472,0.255733,0.304599,1.000000


### helper function

In [8]:
def same_category(_df, i_range, normalize=False):
    df = _df.copy()
    for c in df.columns:
        df[c] = df[c] - 1
        if normalize:
            df[c] = df[c] / (df[c] - 1).max()
        sign = df[c].apply(lambda x: 1 if x > 0 else -1)
        df[c] = (
            df[c].apply(lambda x: 0 if (x > i_range[0]) and (x < i_range[1]) else 1)
            * sign
        )
    return df

In [9]:
def cat_given_cat(cat_df, normalize=True):
    ls = []
    for c in cat_df.columns[1:]:
        for cat in cat_df[c].unique():
            base = cat_df.loc[cat_df[c] == cat]
            sensitivity = base.iloc[:, 0].value_counts().rename(f"{c}_eq_{cat}")
            if normalize:
                sensitivity = sensitivity / len(base)
            ls.append(sensitivity)
    result = pd.concat(ls, axis=1)
    return result

### daily return

In [10]:
def daily_comparrision(coin):
    comp = ana[[coin, "sp500", "nas"]].replace(0, np.nan)
    comp["month"] = (
        pd.Series(comp.index).dt.year * 100 + pd.Series(comp.index).dt.month
    ).values
    comp["year"] = (pd.Series(comp.index).dt.year).values
    comp = comp.dropna()
    for c in [coin, "sp500", "nas"]:
        comp[f"ret_1d_{c}"] = (
            (comp[c] / comp[c].shift(1))
            .replace([np.inf, -np.inf], np.nan)
            .fillna(
                1,
            )
        )
    return comp

In [11]:
comp = daily_comparrision("sol")
comp.iloc[:, -3:].corr()

,ret_1d_sol,ret_1d_sp500,ret_1d_nas
ret_1d_sol,1.000000,0.199161,0.107445
ret_1d_sp500,0.199161,1.000000,0.856776
ret_1d_nas,0.107445,0.856776,1.000000


In [12]:
(comp.iloc[:, -3:] - 1).mean()

ret_1d_sol      0.009336
ret_1d_sp500    0.000472
ret_1d_nas      0.000410
dtype: float64

In [13]:
cat_daily = same_category(comp.iloc[:, -3:], [-0.005, 0.005])

In [14]:
cat_given_cat(cat_daily)

,ret_1d_sp500_eq_0,ret_1d_sp500_eq_-1,ret_1d_sp500_eq_1,ret_1d_nas_eq_0,ret_1d_nas_eq_-1,ret_1d_nas_eq_1
-1,0.144665,0.695652,0.330357,0.112840,0.597701,0.413534
0,0.683544,0.028986,0.071429,0.737354,0.045977,0.037594
1,0.171790,0.275362,0.598214,0.149805,0.356322,0.548872


### monthly return

In [15]:
def monthly_comparrision(coin):
    comp = daily_comparrision(coin)
    comp_month = comp.groupby(by="month")[
        f"ret_1d_{coin}", "ret_1d_sp500", "ret_1d_nas"
    ].prod()
    comp_month.columns = [f"ret_1m_{coin}", "ret_1m_sp500", "ret_1m_nas"]
    return comp_month

In [16]:
comp_month = monthly_comparrision("sol")
comp_month.corr()

/var/folders/y2/kn_782d959q_kshjqj842tpc0000gn/T/ipykernel_16997/53174413.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  comp_month = comp.groupby(by="month")[


,ret_1m_sol,ret_1m_sp500,ret_1m_nas
ret_1m_sol,1.000000,0.367179,0.346062
ret_1m_sp500,0.367179,1.000000,0.910547
ret_1m_nas,0.346062,0.910547,1.000000


In [17]:
(comp_month - 1).mean()

ret_1m_sol      0.520904
ret_1m_sp500    0.018421
ret_1m_nas      0.015499
dtype: float64

In [18]:
cat_monthly = same_category(comp_month, [-0.01, 0.01])

In [19]:
cat_given_cat(cat_monthly)

,ret_1m_sp500_eq_1,ret_1m_sp500_eq_-1,ret_1m_sp500_eq_0,ret_1m_nas_eq_0,ret_1m_nas_eq_1,ret_1m_nas_eq_-1
-1,0.214286,0.75,1.0,0.5,0.111111,0.75
1,0.785714,0.25,NaN,0.5,0.888889,0.25


In [20]:
comp_month.var()

ret_1m_sol      0.904745
ret_1m_sp500    0.001586
ret_1m_nas      0.002575
dtype: float64

### yearly return

In [21]:
def yearly_comparrision(coin):
    comp = daily_comparrision(coin)
    comp_month = comp.groupby(by="year")[
        f"ret_1d_{coin}", "ret_1d_sp500", "ret_1d_nas"
    ].prod()
    comp_month.columns = [f"ret_1y_{coin}", "ret_1y_sp500", "ret_1y_nas"]
    return comp_month

In [22]:
comp_year = yearly_comparrision("sol")
comp_year.corr()

/var/folders/y2/kn_782d959q_kshjqj842tpc0000gn/T/ipykernel_16997/395322584.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  comp_month = comp.groupby(by="year")[


,ret_1y_sol,ret_1y_sp500,ret_1y_nas
ret_1y_sol,1.000000,0.760010,0.553564
ret_1y_sp500,0.760010,1.000000,0.961965
ret_1y_nas,0.553564,0.961965,1.000000


In [23]:
comp_year

,ret_1y_sol,ret_1y_sp500,ret_1y_nas
year,,,
2020,1.520317,1.164798,1.200111
2021,112.456975,1.279096,1.220999
2022,0.739898,0.936373,0.893360


In [24]:
comp_year - 1

,ret_1y_sol,ret_1y_sp500,ret_1y_nas
year,,,
2020,0.520317,0.164798,0.200111
2021,111.456975,0.279096,0.220999
2022,-0.260102,-0.063627,-0.106640


In [25]:
(comp_year - 1).mean()

ret_1y_sol      37.239063
ret_1y_sp500     0.126756
ret_1y_nas       0.104823
dtype: float64

In [26]:
cat_yearly = same_category(comp_year, [-0.02, 0.05])
cat_yearly

,ret_1y_sol,ret_1y_sp500,ret_1y_nas
year,,,
2020,1,1,1
2021,1,1,1
2022,-1,-1,-1


In [27]:
cat_given_cat(cat_yearly)

,ret_1y_sp500_eq_1,ret_1y_sp500_eq_-1,ret_1y_nas_eq_1,ret_1y_nas_eq_-1
-1,NaN,1.0,NaN,1.0
1,1.0,NaN,1.0,NaN


### yearly return for months (yearly returns evaluated each month)

In [28]:
comp_ym = comp_month.copy()
comp_ym = comp_ym.rolling(12).apply(lambda x: np.product(x)).dropna()
# comp_ym = comp_ym - 1

In [29]:
cat_ym = same_category(comp_ym, [-0.02, 0.05])
cat_ym

,ret_1m_sol,ret_1m_sp500,ret_1m_nas
month,,,
202106,1,1,1
202107,1,1,1
202108,1,1,1
202109,1,1,1
202110,1,1,1
202111,1,1,1
202112,1,1,1
202201,1,1,1


In [30]:
cat_given_cat(cat_ym).round(2)

,ret_1m_sp500_eq_1,ret_1m_nas_eq_1
1,1.0,1.0


### magnitude

In [31]:
yret = comp_year - 1
yret

,ret_1y_sol,ret_1y_sp500,ret_1y_nas
year,,,
2020,0.520317,0.164798,0.200111
2021,111.456975,0.279096,0.220999
2022,-0.260102,-0.063627,-0.106640


In [33]:
(yret.ret_1y_sol / yret.ret_1y_sp500)

year
2020      3.157307
2021    399.350233
2022      4.087955
dtype: float64

In [34]:
(yret.ret_1y_sol / yret.ret_1y_nas)

year
2020      2.600149
2021    504.333198
2022      2.439071
dtype: float64